# Tiền xử lí dữ liệu

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Lấy dữ liệu thô


In [2]:
csv_path = '../data/BankChurners.csv'
data = np.genfromtxt(csv_path, delimiter=',', dtype=str, encoding='ascii')

# Loại bỏ hai cột cuối không cần thiết
data = data[:, :-2]
columns = data[0, :]
columns = np.char.replace(columns, '"', '')
data = data[1:, :]
data = np.char.replace(data, '"', '')

In [3]:
# Tạo một từ điển để ánh xạ tên cột với chỉ số của chúng
dict_data = {'CLIENTNUM': 0, 
             'Attrition_Flag': 1,
             'Customer_Age': 2,
             'Gender': 3,
             'Dependent_count': 4, 
             'Education_Level': 5, 
             'Marital_Status': 6,
             'Income_Category': 7, 
             'Card_Category': 8, 
             'Months_on_book': 9,
             'Total_Relationship_Count': 10, 
             'Months_Inactive_12_mon': 11, 
             'Contacts_Count_12_mon': 12,
             'Credit_Limit': 13, 
             'Total_Revolving_Bal': 14, 
             'Avg_Open_To_Buy': 15, 
             'Total_Amt_Chng_Q4_Q1': 16,
             'Total_Trans_Amt': 17, 
             'Total_Trans_Ct': 18, 
             'Total_Ct_Chng_Q4_Q1': 19, 
             'Avg_Utilization_Ratio': 20}

## I. Chọn những đặc trưng cần thiết cho mô hình

### I.1 Bỏ những cột không quan trọng trong việc thiết kế mô hình

Trong quá trình phân tích dữ lệu thì ta sẽ bỏ một số cột dữ liệu với những lí do sau:

1.  **`Avg_Open_To_Buy`**: Vì có tương quan tương quan tuyệt đối **1.00** với `Credit_Limit`. Về mặt toán học, `Avg_Open_To_Buy = Credit_Limit - Revolving_Bal`. Việc giữ cả hai không mang lại thêm thông tin mà gây nhiễu và lãng phí tài nguyên tính toán.

2.  **`Gender`**: Trong biểu đồ tần suất được so với `Attrition_Flag`, cột giá trị phân loại này có tần suất khá giống nhau ở cả hai giá trị M (nam giới) và F (nữ giới) và cũng có tương quan Cramér's V khá thấp với `Attriton_Flag`(**0.01**)


### I.2 Chuyển hóa cột đặc trưng Attrition_Flag và các cột có giá trị phân loại

- Ta sẽ xử lí một số cột giá trị phân loại trước khi chuyển hóa: 
1.  **`Card_Category`**: Vì số lượng thẻ `Blue` chiếm phần lớn nên ta sẽ gộp nhóm (Binning) tất cả các thẻ còn lại (`Sliver`, `Gold`, `Plantinum`) và chuyển hóa thành `0` (Không phải thẻ Blue) và `1`(Có thẻ Blue).
2. **`Marital status`**: Ta sẽ gộp 2 cột `Divorced` và `Single` vì xét theo logic thì nó vẫn có chung một ý nghĩa là hiện tại vẫn chưa có hôn nhân.
- Sau đó ta sẽ áp dụng các thuật toán để chuẩn hóa các cột:
1. `One-hot encoding` cho các cột giá trị phân loại có giá trị Unknown (Vì giá trị này vẫn có tỉ lệ từ `7 - 15%` trong các cột) gồm `Education_Level, Income_Category và Marital_Status`

2. `Binary encoding` cho cột `Attrition_Flag và Card_Category`

In [4]:
def one_hot_np(col):
    uniq = np.unique(col)
    return (col[:, None] == uniq).astype(np.int8), uniq

card_col = data[:, dict_data['Card_Category']]
card_col = (card_col == 'Blue').astype(np.int8)

attrition_col = data[:, dict_data['Attrition_Flag']]
attrition_col = (attrition_col == 'Existing Customer').astype(np.int8)

marital_col = data[:, dict_data['Marital_Status']]
mapped_marital = np.where(
    np.isin(marital_col, ["Divorced", "Single"]), 
    "NotMarried", 
    marital_col
)
marital_ohe, marital_uniques = one_hot_np(mapped_marital)

education_col = data[:, dict_data['Education_Level']]
education_ohe, education_uniques = one_hot_np(education_col)

income_col = data[:, dict_data['Income_Category']]
income_ohe, income_uniques = one_hot_np(income_col)

print("Encoding results:")
print("Marital Status uniques:", marital_uniques)
print("Education Level uniques:", education_uniques)
print("Income Category uniques:", income_uniques)


Encoding results:
Marital Status uniques: ['Married' 'NotMarried' 'Unknown']
Education Level uniques: ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']
Income Category uniques: ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']


## I.3 Chiến thuật Scaling cho các cột có giá trị số


## 1. Tạo cột mới: Avg_Total_Trans

```
Avg_Total_Trans = Total_Trans_Amt / Total_Trans_Ct
```

Mặc dù Total_Trans_Amt và Total_Trans_Ct tương quan mạnh (0.81), nhưng cả hai đều là biến quan trọng. Thay vì loại bỏ, ta tạo ra Avg_Transaction_Value. Biến này có thể giúp mô hình phát hiện ra nhóm khách hàng đặc biệt như "Giao dịch ít nhưng giá trị lớn" (nhóm rủi ro thấp hơn) và nhóm "Giao dịch nhiều nhưng giá trị nhỏ"

---

## 2. Sử dụng Z-Score Scaling cho các nhóm phân phối chuẩn

Dùng khi phân phối chuông, mean & median gần nhau, độ lệch thấp

Scale:

```
x_scaled = (x - mean) / std
```

Những cột sẽ dùng:

* Customer_Age
* Dependent_count
* Total_Relationship_Count
* Months_Inactive_12_mon
* Contacts_Count_12_mon
* Months_on_book
* Total_Amt_Chng_Q4_Q1
* Total_Ct_Chng_Q4_Q1

---

## 3. Sữ dụng Log Scaling cho những cột có phân phối lệch phải, có giá trị lớn

**Dùng log khi:**

* Min > 0
* Max rất lớn
* Median << Max
* Right-skewed

Công thức:

```
x_log = log(1 + x)
```

Cột dùng log:

* Credit_Limit (1438 → 34516, lệch phải mạnh)
* Avg_Open_To_Buy (3 → 34516, lệch phải cực mạnh)
* Total_Revolving_Bal (0 → 2517)
* Avg_Total_Trans (cột mới tạo)


## 4. Sử dụng MinMax cho những cột đã nằm trong phạm vi [0, 1]

Cột:

* Avg_Utilization_Ratio (giá trị 0 → 1, skew thiên về 0)

Nếu muốn scale:

```
x_scaled = (x - x_min) / (x_max - x_min)
```

In [11]:
numerical_score = [
    'Customer_Age', 'Dependent_count', 'Total_Relationship_Count',
    'Months_Inactive_12_mon', 'Contacts_Count_12_mon',
    'Total_Amt_Chng_Q4_Q1', 'Total_Ct_Chng_Q4_Q1',
    'Months_on_book'
]

def zscore(x):
    return (x - x.mean()) / x.std()

numerical_score_col = np.empty((data.shape[0], 0))

for col in numerical_score:
    col_data = data[:, dict_data[col]].astype(float)
    col_z = zscore(col_data)
    numerical_score_col = np.hstack((numerical_score_col, col_z.reshape(-1, 1)))
    

numerical_log = [
    'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy',
    'Avg_Total_Trans']

trans_amt = data[:, dict_data['Total_Trans_Amt']].astype(float)
trans_ct = data[:, dict_data['Total_Trans_Ct']].astype(float)
average_total = trans_amt / trans_ct

def log_scale(x):
    return np.log1p(x)

numerical_log_col = np.empty((data.shape[0], 0))
for col in numerical_log:
    if col == 'Avg_Total_Trans':
        col_data = average_total
    else:
        col_data = data[:, dict_data[col]].astype(float)
    col_log = log_scale(col_data)
    numerical_log_col = np.hstack((numerical_log_col, col_log.reshape(-1, 1)))


def minmax(x):
    return (x - x.min()) / (x.max() - x.min())

util_col = data[:, dict_data['Avg_Utilization_Ratio']].astype(float)
util_mm = minmax(util_col)


In [14]:

final_matrix = np.column_stack([
    attrition_col,          # Target
    card_col,               # Binary Card
    marital_ohe,            # One hot
    education_ohe,
    income_ohe,
    numerical_score_col,                 # Z-score
    numerical_log_col,               # Log-scaled
    util_mm                 # MinMax
])

# ============================================================
#   CREATE HEADERS
# ============================================================


headers = []

headers.append("Attrition_Flag")
headers.append("Is_Blue_Card")

headers += [f"Marital_{u}" for u in marital_uniques]
headers += [f"Education_{u}" for u in education_uniques]
headers += [f"Income_{u}" for u in income_uniques]

headers += [f"{c}_zscore" for c in numerical_score]
headers += [f"{c}_log" for c in numerical_log]

headers.append("Avg_Utilization_Ratio_minmax")


# ============================================================
# 8. SAVE TO CSV USING NUMPY ONLY
# ============================================================

# Combine headers + matrix into one array of strings
header_line = ",".join(headers)
np.savetxt(
    "../data/BankChurners_preprocessed.csv",
    final_matrix,
    delimiter=",",
    header=header_line,
    comments=""
)